uncomment the below cell to download the latest stack overflow dump and upload to gcs

In [ ]:
# %%bash

# sudo apt update && sudo apt install -y p7zip-full

# wget --retry-connrefused --waitretry=1 --read-timeout=20 --timeout=15 -t 0 --continue https://archive.org/download/stackexchange/stackoverflow.com-Posts.7z


# 7z x stackoverflow.com-Posts.7z

# gsutil cp Posts.xml gs://np-training-tmp/stackoverflow/Posts.xml

# ```



In [1]:
import pandas as pd
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

In [2]:
spark = SparkSession\
.builder\
.appName('app')\
.config('spark.jars.packages', 'com.databricks:spark-xml_2.12:0.15.0')\
.config("spark.default.parallelism",200) \
.config("spark.sql.shuffle.partitions",200) \
.getOrCreate()

# .config("spark.sql.crossJoin.enabled",True) \


In [3]:
!gsutil ls -lah  gs://np-training-tmp/stackoverflow/

 89.98 GiB  2022-08-01T18:23:42Z  gs://np-training-tmp/stackoverflow/Posts.xml#1659378222561946  metageneration=1
 17.76 GiB  2022-08-01T09:47:19Z  gs://np-training-tmp/stackoverflow/stackoverflow.com-Posts.7z#1659347239504571  metageneration=1
TOTAL: 2 objects, 115689494677 bytes (107.74 GiB)


In [28]:
posts_input_path = "gs://np-training-tmp/stackoverflow/Posts.xml"
posts_output_path = "gs://np-training-tmp/stackoverflow/posts_processed/"

In [5]:
df = spark \
    .read \
    .format("com.databricks.spark.xml") \
    .option('rowTag', 'row') \
    .option('samplingRatio',0.1) \
    .load(posts_input_path) \
    .repartition(200)

In [6]:
df = df.cache()

In [7]:
df.show(5)

+-----------------+------------+--------------------+-----------+-------------+--------------------+---------------+--------------------+--------------+---+--------------------+--------------------+----------------------+-----------------+-----------------+------------+---------+-----------+------+--------------------+--------------------+----------+
|_AcceptedAnswerId|_AnswerCount|               _Body|_ClosedDate|_CommentCount| _CommunityOwnedDate|_ContentLicense|       _CreationDate|_FavoriteCount|_Id|   _LastActivityDate|       _LastEditDate|_LastEditorDisplayName|_LastEditorUserId|_OwnerDisplayName|_OwnerUserId|_ParentId|_PostTypeId|_Score|               _Tags|              _Title|_ViewCount|
+-----------------+------------+--------------------+-----------+-------------+--------------------+---------------+--------------------+--------------+---+--------------------+--------------------+----------------------+-----------------+-----------------+------------+---------+-----------+--

In [8]:
!ls

bin	   hadoop  libx32      proc				 srv
boot	   home    lost+found  root				 sys
copyright  lib	   media       run				 tmp
dev	   lib32   mnt	       sbin				 usr
etc	   lib64   opt	       sparkmonitor_kernelextension.log  var


In [9]:
df.columns

['_AcceptedAnswerId',
 '_AnswerCount',
 '_Body',
 '_ClosedDate',
 '_CommentCount',
 '_CommunityOwnedDate',
 '_ContentLicense',
 '_CreationDate',
 '_FavoriteCount',
 '_Id',
 '_LastActivityDate',
 '_LastEditDate',
 '_LastEditorDisplayName',
 '_LastEditorUserId',
 '_OwnerDisplayName',
 '_OwnerUserId',
 '_ParentId',
 '_PostTypeId',
 '_Score',
 '_Tags',
 '_Title',
 '_ViewCount']

In [10]:
def cleanup_name(name:str):
    return name[1:]

In [11]:
column_names = [cleanup_name(name) for name in df.columns ] 

In [12]:
len(column_names)

22

In [13]:
df = df.toDF(*column_names)


In [14]:
df.show(5)

+----------------+-----------+--------------------+----------+------------+--------------------+--------------+--------------------+-------------+---+--------------------+--------------------+---------------------+----------------+----------------+-----------+--------+----------+-----+--------------------+--------------------+---------+
|AcceptedAnswerId|AnswerCount|                Body|ClosedDate|CommentCount|  CommunityOwnedDate|ContentLicense|        CreationDate|FavoriteCount| Id|    LastActivityDate|        LastEditDate|LastEditorDisplayName|LastEditorUserId|OwnerDisplayName|OwnerUserId|ParentId|PostTypeId|Score|                Tags|               Title|ViewCount|
+----------------+-----------+--------------------+----------+------------+--------------------+--------------+--------------------+-------------+---+--------------------+--------------------+---------------------+----------------+----------------+-----------+--------+----------+-----+--------------------+---------------

In [15]:
df_qn = df\
    .where(F.col("PostTypeId")==1) \
    .selectExpr("Id","AcceptedAnswerId","Title","Body as QuestionBody","Tags", "ViewCount", "AnswerCount",  "CommentCount" , "Score", "CreationDate") \
    .persist()

In [16]:
df_qn.show(5)

+---+----------------+--------------------+--------------------+--------------------+---------+-----------+------------+-----+--------------------+
| Id|AcceptedAnswerId|               Title|                Body|                Tags|ViewCount|AnswerCount|CommentCount|Score|        CreationDate|
+---+----------------+--------------------+--------------------+--------------------+---------+-----------+------------+-----+--------------------+
|  4|               7|How to convert a ...|<p>I want to use ...|<c#><floating-poi...|    65143|         13|           4|  763|2008-07-31 21:42:...|
|  6|              31|Why did the width...|<p>I have an abso...|<html><css><inter...|    22702|          7|           0|  314|2008-07-31 22:08:...|
|  9|            1404|How do I calculat...|<p>Given a <code>...|<c#><.net><datetime>|   741405|         70|          10| 2120|2008-07-31 23:40:...|
| 11|            1248|Calculate relativ...|<p>Given a specif...|<c#><datetime><ti...|   190699|         40|     

In [ ]:
df_qn.count()

22634239

In [ ]:
df_answer = df\
    .where(F.col("PostTypeId")==2) \
    .selectExpr("Id","Body as AnswerBody") \
    .persist()


In [ ]:
df_answer.show(5)

+---+--------------------+
| Id|          AnswerBody|
+---+--------------------+
|  7|<p>An explicit ca...|
| 12|<p>Here's how I d...|
| 18|<p>For a table li...|
| 22|<p>The best way t...|
| 26|<p>The answer by ...|
+---+--------------------+
only showing top 5 rows



In [ ]:
df_answer.count()

33520483

In [ ]:
?df_qn.join

In [ ]:
df_final = df_qn.join(df_answer
    , on=[ 
        df_qn.AcceptedAnswerId == df_answer.Id
    ]
    , how="inner"
    )

In [34]:
df_final = df_qn.alias("qn").join(df_answer.alias("answer")
    , on=[ 
        F.col("qn.AcceptedAnswerId") == F.col("answer.Id") 
    ]
    , how="inner"
    ) \
    .drop(F.col("answer.Id"))

In [35]:
df_final.show(5)

+-----+----------------+--------------------+--------------------+--------------------+---------+-----------+------------+-----+--------------------+--------------------+
|   Id|AcceptedAnswerId|               Title|                Body|                Tags|ViewCount|AnswerCount|CommentCount|Score|        CreationDate|          AnswerBody|
+-----+----------------+--------------------+--------------------+--------------------+---------+-----------+------------+-----+--------------------+--------------------+
|   17|              26|Binary Data in MySQL|<p>How do I store...|<mysql><database>...|    83953|          9|           3|  194|2008-08-01 05:09:...|<p>The answer by ...|
| 2873|            2927|Choosing a static...|<p>I'm working on...|  <c><unix><testing>|    55970|         15|           3|   69|2008-08-05 21:19:...|<p>Don't overlook...|
| 7224|            7225|Change the width ...|<p>Is it possible...| <vb.net><scrollbar>|    16007|          2|           0|    7|2008-08-10 15:48:

In [36]:
df_final.count()

11588225

In [ ]:
df_final.repartition(30).write.mode("overwrite").parquet(posts_output_path)

In [38]:
!ls

bin	   hadoop  libx32      proc				 srv
boot	   home    lost+found  root				 sys
copyright  lib	   media       run				 tmp
dev	   lib32   mnt	       sbin				 usr
etc	   lib64   opt	       sparkmonitor_kernelextension.log  var
